In [11]:
# Import des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import time
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm

import fonctions
from fonctions import scan_xml, modif_chemins_declaration

# options
pd.set_option('display.max_row', 200)
pd.set_option('display.max_columns', 50)

# dossier
folder = 'D:/Fichiers_dev/test_xml/tables/'
folder2 = 'D:/Fichiers_dev/test_xml/'

data3 = pd.read_csv(folder + 'data_nettoyees.csv')

In [12]:
data3.head()

,id_declaration,chemins,champs,valeurs
0,4344aaa1-874d-4e6d-9b1a-45f7725b710c,dateDepot,dateDepot,11/07/2022 15:40:13
1,4344aaa1-874d-4e6d-9b1a-45f7725b710c,uuid,uuid,4344aaa1-874d-4e6d-9b1a-45f7725b710c
2,4344aaa1-874d-4e6d-9b1a-45f7725b710c,origine,origine,ADEL
3,4344aaa1-874d-4e6d-9b1a-45f7725b710c,complete,complete,true
4,4344aaa1-874d-4e6d-9b1a-45f7725b710c,attachedFiles/attachedFiles/fileName,fileName,VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML


In [13]:
# champs de la table en cours
champs_declaration = pd.read_excel(folder2 + 'chemins_uniques_2.xlsx', sheet_name='declarations')
champs_declaration = champs_declaration['champs'].str.replace('declaration/',"").values
champs_declaration

array(['complete', 'dateDepot', 'declarationVersion',
       'evenementMajeurDto/neant', 'general/chiffreAffaire',
       'general/dateDebutMandat', 'general/declarationModificative',
       'general/mandat/label', 'general/nomSociete',
       'general/nomSocieteMere', 'general/organe/codeListeOrgane',
       'general/organe/codeOrgane', 'general/organe/labelDeclaration',
       'general/organe/labelOrgane', 'general/organe/nomListeOrgane',
       'general/organe/organeParent', 'general/qualiteDeclarant',
       'general/qualiteDeclarantForPDF',
       'general/qualiteMandat/codCategorieMandat',
       'general/qualiteMandat/codeListeOrgane',
       'general/qualiteMandat/codTypeMandatFichier',
       'general/qualiteMandat/labelOrgane',
       'general/qualiteMandat/labelTypeMandat',
       'general/qualiteMandat/nomCategorieMandat',
       'general/qualiteMandat/typeMandat', 'general/regimeMatrimonial',
       'general/regimeMatrimonialComments', 'general/typeDeclaration/id',
       

In [14]:
declarations = data3.copy()

# mask de la table déclaration, mapping entre les champs de la table et le chemin xml
mask_declaration = declarations['chemins'].isin(champs_declaration)

declarations = declarations[mask_declaration][['id_declaration', 'chemins', 'valeurs']]

In [15]:
# modification du chemin avec la fonction modif_chemins_declaration
declarations['chemins'] = declarations['chemins'].apply(modif_chemins_declaration)

In [16]:
# suppression des doublons
declarations = declarations.drop(declarations[declarations.duplicated(subset=['id_declaration', 'chemins'],
                                                                      keep='last')].index)

In [17]:
# pivot des champs
declarations = declarations.pivot(index='id_declaration', columns='chemins', values='valeurs').reset_index()

In [ ]:
# creation de la clé étrangère id_individu
declarations['id_individu'] = (declarations['declarant_nom']+declarations['declarant_prenom']+
                               declarations['declarant_date_naissance'])


# suppression des colonnes liées à la personne
declarations = declarations.drop(columns=['declarant_nom', 'declarant_prenom', 'declarant_date_naissance'])


# encodage id_declaration pour avoir un id en auto increment
encoder_id_declaration = LabelEncoder()
declarations['id_declaration'] = encoder_id_declaration.fit_transform(declarations['id_declaration'])


# encodage id_individus
declarations['id_individu'] = encoder_id_individu.transform(declarations['id_individu'])

# export en csv
declarations.to_csv(folder + 'declarations.csv', index=False)